# Первичный анализ данных

In [ ]:
%load_ext autoreload
%autoreload 2

import os 

PROJECT_DPATH = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_DPATH = os.path.join(PROJECT_DPATH, "data")

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms
import seaborn as sns

from mnist_recognition.transforms import Invertor

RANDOM_SEED = 42 
torch.backends.cudnn.enabled = False
torch.manual_seed(RANDOM_SEED)
torch.cuda.seed()

## Загрузка данных

In [ ]:
transform = transforms.Compose([Invertor()])

# загружаем обучающую выборку 
train_data = torchvision.datasets.MNIST(
    DATA_DPATH, train=True, transform=transform, download=True
)

# разделяем обучающую выборку на обучающую и валидационную выборки
# 70% для обучения, 30% для валидации
train_size = int(len(train_data) * 0.7)
valid_size = len(train_data) - train_size
train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    DATA_DPATH, train=False, transform=transform, download=True
)

In [ ]:
# генерируем имена колонок для пикселей изображения
img_col_names = [f"{i}x{j}" for i in range(1, 29) for j in range(1, 29)]

In [ ]:
train_df = []
stream = tqdm(train_data, desc="Train Data Processing")
for img, label in stream:
    np_img = np.array(img)
    train_df.append([label] + np_img.flatten().tolist())

train_df = pd.DataFrame(train_df, columns=["label"] + img_col_names)
train_df.shape

In [ ]:
valid_df = []
stream = tqdm(valid_data, desc="Valid Data Processing")
for img, label in stream:
    np_img = np.array(img)
    valid_df.append([label] + np_img.flatten().tolist())

valid_df = pd.DataFrame(valid_df, columns=["label"] + img_col_names)
valid_df.shape

In [ ]:
test_df = []
stream = tqdm(test_data, desc="Test Data Processing")
for img, label in stream:
    np_img = np.array(img)
    test_df.append([label] + np_img.flatten().tolist())

test_df = pd.DataFrame(test_df, columns=["label"] + img_col_names)
test_df.shape

## Анализ обучающей выборки

In [ ]:
# Общая информация о данных: размер датасета, тип данных
train_df.info()

In [ ]:
# Посмотрим на то как представленны данные 
train_df.head()

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(train_df['label'].unique())

Обучающая выборка содержит 42,000 строк, 785 колонок. 

* Первая колонка- лэйбл картинки, содержит значения от 0-9. 

* Колонки от 1-785 (1х1 - 28х28) сосдержат значения каждого пикселя изображения. 

Т.е в датасете представлено 42,000 примеров изображения для обучения 

Рассмотрим информацию о данных для каждой колонки отдельно для того, чтобы оценить есть ли пропуски в данных 

In [ ]:
train_df.isnull().sum().sum()

Пропуски в данных отсутсвуют 

Рассмотрим соотношение значений в целевой колонке - колонке лэйблов изображений
Представим информацию в числовом виде и в виде графика 

In [ ]:
plt.figure(figsize=(10, 5)) 
_fig = sns.histplot(train_df.label, bins=10)

_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

plt.ylabel("Количество изображений")
plt.xlabel("Истинные классы")
plt.title("Распределение целевых классов в обучающей выборке")


plt.tight_layout()
plt.show()

Дисбаланса в данных нет, датасет содержит примерно равное количество примеров изображений по каждой цифре

Обозначим среднее значение пикселя в изображении для разных цифр как средняя интенсивность.

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_train_labels=pd.DataFrame(
    {
        "labels": train_df['label'],
        # make backgroud = 0
        "intensity": abs(train_df.iloc[:,1:] - 255).mean(axis=1)
    }
)

In [ ]:
plt.figure(figsize=(10, 5))
_fig = sns.barplot(x=df_train_labels["labels"], y=df_train_labels["intensity"])
_fig.bar_label(_fig.containers[0])

plt.title("Распределение средней интенсивности пикселей")
plt.xlabel("Истинные классы")
plt.ylabel("Средняя интенсивность")

plt.tight_layout()
plt.show()

Из данного графика видно, что самая низкая интенсивность наблюдается у цифры 1 (что связано с особенностями ее написания), самой высокой интенсивностью обладают цифры 0 и 8. 

Графики распределения средней интенсивности по лэйблу

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))

counter = 0
for row_i in range(2):
    for col_i in range(5):
        sns.histplot(
            ax=ax[row_i][col_i], 
            data=df_train_labels['intensity'][df_train_labels['labels']==counter]
        )        
        ax[row_i][col_i].set_xlabel("Интенсивность")
        ax[row_i][col_i].set_ylabel("Количество пикселей")
        ax[row_i][col_i].set_title(f"Класс {counter}")
        counter += 1 

plt.tight_layout()
plt.show()

Полученные распределения близки к нормальному распределению, но некоторые графики имеют более высокую дисперсию, чем другие, так как написание цифр у разных людей отличается. 

Так, например, у 1 наименьший разброс, в связи с чем можно предположить, что написание 1 имеет меньше всего различий.

Посмотрим на выборочные изображения рукописных цифр 

In [ ]:
@interact
def show_digits(label=widgets.IntSlider(value=0, min=0, max=9)):
    _fig, ax = plt.subplots(nrows=2, ncols=5, figsize=[25, 10])
    for row_i in range(2):
        for col_i in range(5):
            j = np.random.choice(train_df[train_df['label'] == label].index)
            digit = np.array(train_df.loc[j, train_df.columns != "label"]).reshape(28, 28)
            
            ax[row_i][col_i].imshow(digit, cmap="gray")
    plt.tight_layout()
    plt.show()

## Анализ валидационной выборки

In [ ]:
# Общая информация о данных: размер датасета, тип данных
valid_df.info()

In [ ]:
valid_df.head()

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(valid_df['label'].unique())

In [ ]:
valid_df.isnull().sum().sum()

Данные выборки для валидации имеют такое же представление как и выборка для обучения: 

* первая колонка - это лэйбл (класс) картинки, она содержит значения от 0-9, 
* колонки от 1-785 (1х1 - 28х28) сосдержат значения каждого пиксля изображения.

Валидационная выбрка содержит 18,000 изображений.

Пропуски в данных отсутвуют 

In [ ]:
plt.figure(figsize=(10, 5)) 
_fig = sns.histplot(valid_df.label, bins=10)

_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

plt.ylabel("Количество изображений")
plt.xlabel("Истинные классы")
plt.title("Распределение целевых классов в валидационной выборке")


plt.tight_layout()
plt.show()

Выборка содержит примерно равное количество изображений для каждого лэйбла

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_valid_labels = pd.DataFrame(
    {
        'labels':valid_df['label'],
        # make backgroud = 0
        "intensity": abs(valid_df.iloc[:,1:] - 255).mean(axis=1)
    }
)

In [ ]:
plt.figure(figsize=(10, 5))
_fig = sns.barplot(x=df_valid_labels["labels"], y=df_valid_labels["intensity"])
_fig.bar_label(_fig.containers[0])

plt.title("Распределение средней интенсивности пикселей")
plt.xlabel("Истинные классы")
plt.ylabel("Средняя интенсивность")

plt.tight_layout()
plt.show()

Цифры 0 и 8 имеют наибольшие значения средней интенсивности изображений, а цифры 1 и 4 имеют самые низкие показатели интенсивности

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))

counter = 0
for row_i in range(2):
    for col_i in range(5):
        sns.histplot(
            ax=ax[row_i][col_i], 
            data=df_valid_labels['intensity'][df_valid_labels['labels']==counter]
        )        
        ax[row_i][col_i].set_xlabel("Интенсивность")
        ax[row_i][col_i].set_ylabel("Количество пикселей")
        ax[row_i][col_i].set_title(f"Класс {counter}")
        counter += 1 

plt.tight_layout()
plt.show()

Дисперсия распределения интенсивности зависит от того насколько различается написание цифр

Примеры изображений из валидационной выборки

In [ ]:
@interact
def show_digits(label=widgets.IntSlider(value=0, min=0, max=9)):
    _fig, ax = plt.subplots(nrows=2, ncols=5, figsize=[25, 10])
    for row_i in range(2):
        for col_i in range(5):
            j = np.random.choice(valid_df[valid_df['label'] == label].index)
            digit = np.array(valid_df.loc[j, valid_df.columns != "label"]).reshape(28, 28)
            
            ax[row_i][col_i].imshow(digit, cmap="gray")
    plt.tight_layout()
    plt.show()

## Анализ тестовой выборки

In [ ]:
# Общая информация о данных: размер датасета, тип данных
test_df.info()

In [ ]:
test_df.head(3)

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(test_df['label'].unique())

In [ ]:
# Кол-во пропусков в данных
test_df.isnull().sum().sum()

Тестовая выбрка содержит 10,000 изображений.
Пропуски в данных отсутсвуют

In [ ]:
plt.figure(figsize=(10, 5)) 
_fig = sns.histplot(test_df.label, bins=10)

_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

plt.ylabel("Количество изображений")
plt.xlabel("Истинные классы")
plt.title("Распределение целевых классов в тестовой выборке")


plt.tight_layout()
plt.show()

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_test_labels = pd.DataFrame(
    {
        'labels':test_df['label'],
        # make backgroud = 0
        "intensity": abs(test_df.iloc[:,1:] - 255).mean(axis=1)
    }
)

In [ ]:
plt.figure(figsize=(10, 5))
_fig = sns.barplot(x=df_test_labels["labels"], y=df_test_labels["intensity"])
_fig.bar_label(_fig.containers[0])

plt.title("Распределение средней интенсивности пикселей")
plt.xlabel("Истинные классы")
plt.ylabel("Средняя интенсивность")

plt.tight_layout()
plt.show()

В тестовой выборке средняя интенсивность изображений:
* цифры 2 стала выше чем в обучающей и валидационной выборке,
* средняя интенсивность цифры 7 меньше, чем в обущающей и валидационной выборках
* наибольша интенсивность у цифр 0,2,8;
* наименьшая интенсивность у цифр 1,7

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))

counter = 0
for row_i in range(2):
    for col_i in range(5):
        sns.histplot(
            ax=ax[row_i][col_i], 
            data=df_test_labels['intensity'][df_test_labels['labels']==counter]
        )        
        ax[row_i][col_i].set_xlabel("Интенсивность")
        ax[row_i][col_i].set_ylabel("Количество пикселей")
        ax[row_i][col_i].set_title(f"Класс {counter}")
        counter += 1 

plt.tight_layout()
plt.show()

Распределения средней интенсивности цифр 2, 3, 4 имеют несколько вершин, что может говорить о том, что для данных цифр имеется несколько различных вариантов их написания.

Примеры изображений цифр из тестовой выборки

In [ ]:
@interact
def show_digits(label=widgets.IntSlider(value=0, min=0, max=9)):
    _fig, ax = plt.subplots(nrows=2, ncols=5, figsize=[25, 10])
    for row_i in range(2):
        for col_i in range(5):
            j = np.random.choice(test_df[test_df['label'] == label].index)
            digit = np.array(test_df.loc[j, test_df.columns != "label"]).reshape(28, 28)
            
            ax[row_i][col_i].imshow(digit, cmap="gray")
    plt.tight_layout()
    plt.show()

## Сравнительные распределения

In [ ]:
_fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

_fig = sns.histplot(train_df.label, bins=10, ax=ax[0])
_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

_fig.set_ylabel("Количество изображений")
_fig.set_xlabel("Истинные классы")
_fig.set_title("Распределение целевых классов в обучающей выборке")

_fig = sns.histplot(valid_df.label, bins=10, ax=ax[1])
_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

_fig.set_ylabel("Количество изображений")
_fig.set_xlabel("Истинные классы")
_fig.set_title("Распределение целевых классов в валидационной выборке")


_fig = sns.histplot(test_df.label, bins=10, ax=ax[2])
_fig.bar_label(_fig.containers[0])
_fig.set_xticks(list(range(0, 10)))

_fig.set_ylabel("Количество изображений")
_fig.set_xlabel("Истинные классы")
_fig.set_title("Распределение целевых классов в тестовой выборке")


plt.tight_layout()
plt.show()

In [ ]:
_fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

_fig = sns.barplot(x=df_train_labels["labels"], y=df_train_labels["intensity"], ax=ax[0])
_fig.bar_label(_fig.containers[0])

_fig.set_title("Распределение средней интенсивности пикселей\nОбучающая выборка")
_fig.set_xlabel("Истинные классы")
_fig.set_ylabel("Средняя интенсивность")

_fig = sns.barplot(x=df_valid_labels["labels"], y=df_valid_labels["intensity"], ax=ax[1])
_fig.bar_label(_fig.containers[0])

_fig.set_title("Распределение средней интенсивности пикселей\nВалидационная выборка")
_fig.set_xlabel("Истинные классы")
_fig.set_ylabel("Средняя интенсивность")

_fig = sns.barplot(x=df_test_labels["labels"], y=df_test_labels["intensity"], ax=ax[2])
_fig.bar_label(_fig.containers[0])

_fig.set_title("Распределение средней интенсивности пикселей\nТестовая выборка")
_fig.set_xlabel("Истинные классы")
_fig.set_ylabel("Средняя интенсивность")

plt.tight_layout()
plt.show()